In [1]:
import torch
import torch.nn as nn
import numpy as np
import imageio.v2 as imageio
import cv2
import argparse
import json
import os
import time
from tqdm import tqdm, trange

In [2]:
args = {
    'config': 'configs/lego.txt',
    'expname': 'blender_paper_lego',
    'basedir': './logs',
    'datadir': './data/nerf_synthetic/lego',
    'netdepth': 8,
    'netwidth': 256,
    'netdepth_fine': 8,
    'netwidth_fine': 256,
    'N_rand': 1024,
    'lrate': 0.0005,
    'lrate_decay': 500,
    'chunk': 32768,
    'netchunk': 65536,
    'no_batching': True,
    'no_reload': False,
    'ft_path': None,
    'N_samples': 64,
    'N_importance': 128,
    'perturb': 1.0,
    'use_viewdirs': True,
    'i_embed': 0,
    'multires': 10,
    'multires_views': 4,
    'raw_noise_std': 0.0,
    'render_only': False,
    'render_test': False,
    'render_factor': 0,
    'precrop_iters': 500,
    'precrop_frac': 0.5,
    'dataset_type': 'blender',
    'testskip': 8,
    'shape': 'greek',
    'white_bkgd': True,
    'half_res': True,
    'factor': 8,
    'no_ndc': False,
    'lindisp': False,
    'spherify': False,
    'llffhold': 8,
    'i_print': 100,
    'i_img': 500,
    'i_weights': 10000,
    'i_testset': 50000,
    'i_video': 50000
}
args = argparse.Namespace(**args)